In [ ]:
#default_exp notebook.showdoc

In [ ]:
# export
from fastai_local.core import *
from fastai_local.imports import *
from fastai_local.data.pipeline import *
from fastai_local.data.external import *
from fastai_local.test import *
from fastai_local.notebook.export import *
import inspect,enum
from IPython.display import Markdown,display

# Show doc
> Functions to show the doc cells in notebooks

In [ ]:
test_cases = [
    listify,    #Basic func
    Pipeline,   #Basic class
    ConfigKey,  #Enum
    compose,    #Func with star args and type annotation
    untar_data, #Func with defaults
    add_docs    #Func with kwargs
]

## Gather the information

### Names and stuff

In [ ]:
def fn_name(ft):
    "Get the name of `ft`"
    if hasattr(ft, '__name__'):       return ft.__name__
    elif getattr(ft, '_name', False): return ft._name
    elif hasattr(ft,'__origin__'):    return str(ft.__origin__).split('.')[-1] #types
    else:                             return str(ft).split('.')[-1]

In [ ]:
test_eq([fn_name(o) for o in test_cases], 
        ['listify', 'Pipeline', 'ConfigKey', 'compose', 'untar_data', 'add_docs'])
test_eq(fn_name(Union[Tensor, float]), 'Union')

In [ ]:
def get_anchor(fn):
    "Anchor name associated to `fn`"
    if hasattr(fn,'__qualname__'): return fn.__qualname__
    if inspect.ismethod(fn): return fn_name(fn.__self__) + '.' + fn_name(fn)
    return fn_name(fn)

In [ ]:
test_eq([get_anchor(o) for o in test_cases], 
        ['listify', 'Pipeline', 'ConfigKey', 'compose', 'untar_data', 'add_docs'])
test_eq(get_anchor(Pipeline.composed), 'Pipeline.composed')

The inspect module lets us know quickly if an object is a function or a class but it doesn't distinguish classes and enums.

In [ ]:
def is_enum(cls): 
    "Check if `cls` is an enum or another type of class"
    return cls.__class__ == enum.Enum or cls.__class__ == enum.EnumMeta

In [ ]:
assert is_enum(ConfigKey)
assert not is_enum(Pipeline)

In [ ]:
def _get_mod_name(ft): 
    "Return the name of the module where `ft` is defined"
    return inspect.getmodule(ft).__name__

In [ ]:
test_eq(_get_mod_name(Pipeline), 'fastai_local.data.pipeline')
test_eq(_get_mod_name(untar_data), 'fastai_local.data.external')
test_eq(_get_mod_name(ConfigKey), 'fastai_local.data.external')

### Links

In [ ]:
#hide
#Tricking jupyter notebook to have a __file__ attribute. All _file_ will be replaced by __file__
_file_ = Path('fastai_local').absolute()/'notebook'/'show_doc.py'

In [ ]:
def _get_pytorch_index():
    if not (Path(_file_).parent/'index_pytorch.txt').exists(): return {}
    return json.load(open(Path(_file_).parent/'index_pytorch.txt', 'r'))

def add_pytorch_index(func_name, url):
    "Add `func_name` in the PyTorch index for automatic links."
    index = _get_pytorch_index()
    if not url.startswith("https://pytorch.org/docs/stable/"):
        url = "https://pytorch.org/docs/stable/" + url
    index[func_name] = url
    json.dump(index, open(Path(_file_).parent/'index_pytorch.txt', 'w'), indent=2)

`url` can be the full url or just the part after `https://pytorch.org/docs/stable/`, see the example below.

In [ ]:
#hide
ind,ind_bak = Path(_file_).parent/'index_pytorch.txt',Path(_file_).parent/'index_pytorch.bak'
if ind.exists(): shutil.move(ind, ind_bak)
test_eq(_get_pytorch_index(), {})
add_pytorch_index('Tensor', 'tensors.html#torch-tensor')
test_eq(_get_pytorch_index(), {'Tensor':'https://pytorch.org/docs/stable/tensors.html#torch-tensor'})
if ind_bak.exists(): shutil.move(ind_bak, ind)

In [ ]:
add_pytorch_index('Tensor', 'tensors.html#torch-tensor')
add_pytorch_index('device', 'tensor_attributes.html#torch-device')
add_pytorch_index('DataLoader', 'data.html#torch.utils.data.DataLoader')

In [ ]:
def is_fastai_module(name):
    "Test if `name` is a fastai module."
    dir_name = os.path.sep.join(name.split('.'))
    return (Path(_file_).parent.parent/f"{dir_name}.py").exists()

In [ ]:
assert is_fastai_module('data.external')
assert is_fastai_module('core')
assert not is_fastai_module('export')

In [ ]:
#Might change once the library is renamed fastai.
def _is_fastai_class(ft): return belongs_to_module(ft, 'fastai_source')
def _strip_fastai(s): return re.sub(r'^fastai_local\.', '', s)
FASTAI_DOCS = '' #TODO: change when live

In [ ]:
def doc_link(name, include_bt:bool=True):
    "Create link to documentation for `name`."
    cname = f'`{name}`' if include_bt else name
    if is_fastai_module(name): return f'[{cname}]({FASTAI_DOCS}/{name}.html)'
    try_fastai = source_nb(name, is_name=True)
    if try_fastai:
        page = '.'.join(try_fastai.split('_')[1:]).replace('.ipynb', '.html')
        return f'[{cname}]({FASTAI_DOCS}/{page}#{name})'
    try_pytorch = _get_pytorch_index().get(name, None)
    if try_pytorch: return f'[{cname}]({try_pytorch})'
    return cname

In [ ]:
test_eq(doc_link('data.pipeline'), f'[`data.pipeline`]({FASTAI_DOCS}/data.pipeline.html)')
test_eq(doc_link('Pipeline'), f'[`Pipeline`]({FASTAI_DOCS}/data.pipeline.html#Pipeline)')
test_eq(doc_link('Transform.create'), 
        f'[`Transform.create`]({FASTAI_DOCS}/data.pipeline.html#Transform.create)')
test_eq(doc_link('Tensor'), '[`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)')
test_eq(doc_link('Tenso'), '`Tenso`')

In [ ]:
def add_doc_links(text):
    "Try to find doc link for any item between backticks in `text`."
    pat = re.compile("\[`([^`]*)`\](?:\([^)]*\))|`([^`]*)`")
    def _replace_link(m): return doc_link(m.group(1) or m.group(2))
    return re.sub(pat, _replace_link, text)

This function not only add links to backstick keywords, it also update the links that are already in the text.

In [ ]:
test_eq(add_doc_links('This is an example of `Pipeline`'), 
        "This is an example of [`Pipeline`](/data.pipeline.html#Pipeline)")
test_eq(add_doc_links('Here we alread add a link in [`Tensor`](fake)'), 
        "Here we alread add a link in [`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)")

## Show documentation

In [ ]:
def partial_repr(t):
    args = (t.func.__name__,) + t.args + tuple([f'{k}={v}' for k,v in t.keywords.items()])
    reprs = ', '.join([doc_link(o) for o in args])
    return f'<code>partial(</code>{reprs}<code>)</code>'

In [ ]:
test_eq(partial_repr(partial(untar_data, c_key=ConfigKey.Model)),
        "<code>partial(</code>[`untar_data`](/data.external.html#untar_data), `c_key=3`<code>)</code>")

In [ ]:
def type_repr(t):
    "Representation of type `t` (in a type annotation)"
    if getattr(t, '__args__', None):
        args = t.__args__
        if len(args)==2 and args[1] == type(None):
            return f'`Optional`\[{type_repr(args[0])}\]'
        reprs = ', '.join([type_repr(o) for o in args])
        return f'{doc_link(fn_name(t))}\[{reprs}\]'
    else: return doc_link(fn_name(t))

In [ ]:
test_eq(type_repr(Optional[Tensor]), 
        '`Optional`\\[[`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)\\]')
test_eq(type_repr(Union[Tensor, float]), 
        '`Union`\\[[`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor), `float`\\]')

In [ ]:
_arg_prefixes = {inspect._VAR_POSITIONAL: '\*', inspect._VAR_KEYWORD:'\*\*'}

def format_param(p):
    "Formats function param to `param1:Type=val`. Font weights: param1=bold, val=bold+italic"
    arg_prefix = _arg_prefixes.get(p.kind, '') # asterisk prefix for *args and **kwargs
    res = f"**{arg_prefix}`{p.name}`**"
    if hasattr(p, 'annotation') and p.annotation != p.empty: res += f':{type_repr(p.annotation)}'
    if p.default != p.empty:
        default = getattr(p.default, 'func', p.default)
        default = getattr(default, '__name__', default)
        if is_enum(default.__class__):
            res += f'=*`{default.__class__.__name__}.{default.name}`*'
        else: res += f'=*`{repr(default)}`*'
    return res

In [ ]:
sig = inspect.signature(untar_data)
params = [format_param(p) for _,p in sig.parameters.items()]
test_eq(params, [
    '**`url`**',
    '**`fname`**=*`None`*',
    '**`dest`**=*`None`*',
    '**`c_key`**=*`ConfigKey.Data`*',
    '**`force_download`**=*`False`*'])

sig = inspect.signature(compose)
params = [format_param(p) for _,p in sig.parameters.items()]
test_eq(params[0], '**\\*`funcs`**:`Callable`')

In [ ]:
def _format_enum_doc(enum, full_name:str)->str:
    "Formatted `enum` definition to show in documentation"
    vals = ', '.join(enum.__members__.keys())
    return f'<code>{full_name}</code>',f'<code>Enum</code> = [{vals}]'

In [ ]:
test_eq(_format_enum_doc(ConfigKey, 'ConfigKey'),
        ('<code>ConfigKey</code>', '<code>Enum</code> = [Data, Archive, Model]'))

In [ ]:
def _format_func_doc(func, full_name:str=None)->str:
    "Format `func` definition to show in documentation"
    sig = inspect.signature(func)
    name = f'<code>{full_name or func.__name__}</code>'
    fmt_params = [format_param(param) for name,param
                  in sig.parameters.items() if name not in ('self','cls')]
    arg_str = f"({', '.join(fmt_params)})"
    f_name = f"<code>class</code> {name}" if inspect.isclass(func) else name
    return f'{f_name}',f'{name}{arg_str}'

In [ ]:
test_eq(_format_func_doc(listify), ('<code>listify</code>', '<code>listify</code>(**`o`**)'))

In [ ]:
def _format_cls_doc(cls, full_name:str)->str:
    "Format `cls` definition to show in documentation"
    parent_class = inspect.getclasstree([cls])[-1][0][1][0]
    name,args = _format_func_doc(cls, full_name)
    if parent_class != object: args += f' :: {doc_link(fn_name(parent_class))}'
    return name,args

In [ ]:
test_eq(_format_cls_doc(Pipeline, 'Pipeline'), ('<code>class</code> <code>Pipeline</code>',
                                    '<code>Pipeline</code>(**`tfms`**)'))

In [ ]:
def show_doc(elt, doc_string=True, full_name=None, title_level=None, alt_doc_string='', markdown=True):
    "Show documentation for element `elt`. Supported types: class, function, and enum."
    anchor_id = get_anchor(elt)
    elt = getattr(elt, '__func__', elt)
    full_name = full_name or fn_name(elt)
    if inspect.isclass(elt):
        if is_enum(elt.__class__):   name,args = _format_enum_doc(elt, full_name)
        else:                        name,args = _format_cls_doc (elt, full_name)
    elif isinstance(elt, Callable):  name,args = _format_func_doc(elt, full_name)
    else: raise Exception(f'doc definition not supported for {full_name}')
    #TODO: source
    link = 'https://github.com/fastai/fastai/'
    source_link = '<a href="{link}" class="source_link" style="float:right">[source]</a>'
    title_level = ifnone(title_level, 3 if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{anchor_id}" class="doc_header">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n'
    if doc_string and inspect.getdoc(elt): doc += add_doc_links(inspect.getdoc(elt))
    if markdown: display(Markdown(doc))
    else: return doc

In [ ]:
show_doc(listify)

<h4 id="listify" class="doc_header"><code>listify</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>listify</code>(**`o`**)

Make `o` a list.

In [ ]:
show_doc(Pipeline)

<h3 id="Pipeline" class="doc_header"><code>class</code> <code>Pipeline</code><a href="{link}" class="source_link" style="float:right">[source]</a></h3>

> <code>Pipeline</code>(**`tfms`**)

A pipeline of composed (for encode/decode) transforms, setup one at a time

In [ ]:
show_doc(ConfigKey)

<h3 id="ConfigKey" class="doc_header"><code>class</code> <code>ConfigKey</code><a href="{link}" class="source_link" style="float:right">[source]</a></h3>

> <code>ConfigKey</code>(**`value`**, **`names`**=*`None`*, **`module`**=*`None`*, **`qualname`**=*`None`*, **`type`**=*`None`*, **`start`**=*`1`*) :: `IntEnum`

An enumeration.

In [ ]:
show_doc(compose)

<h4 id="compose" class="doc_header"><code>compose</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>compose</code>(**\*`funcs`**:`Callable`)

Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to all

In [ ]:
show_doc(untar_data)

<h4 id="untar_data" class="doc_header"><code>untar_data</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>untar_data</code>(**`url`**, **`fname`**=*`None`*, **`dest`**=*`None`*, **`c_key`**=*`ConfigKey.Data`*, **`force_download`**=*`False`*)

Download `url` to `fname` if `dest` doesn't exist, and un-tgz to folder `dest`.

In [ ]:
show_doc(add_docs)

<h4 id="add_docs" class="doc_header"><code>add_docs</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>add_docs</code>(**\*\*`docs`**)

Copy values from [`docs`](/core.html#docs) to `cls` docstrings, and confirm all public methods are documented